# test af både jupter og det med at importere ting som jeg har tænkt mig

TODO:
Fundet ud af hvad jeg gør med xF og de andre for at forkorte dem(måske ændre i read binary?)

In [1]:
from data.read_binary import readDNSdata
import os
import dask.array as da
import dask
from dask.distributed import Client
import xarray as xr

raw_path="../data/raw/"
iterim_path='../data/interim/'


filelist = os.listdir(raw_path)
filelist=sorted(filelist)
client = Client()


In [2]:
for i in range(3):
    filelist[i]=raw_path+filelist[i]


In [3]:
x=client.map(readDNSdata,filelist)


In [7]:
quantities, quanList, xF, yF, zF, length, storl, paraString = x[0].result()

In [8]:
xF=xF[:-1]
zF=zF[:-1]
ds=xr.Dataset(
    {
        "u_vel": (['x','y','z'],quantities[0]),
        "v_vel": (['x','y','z'],quantities[1]),
        "w_vel": (['x','y','z'],quantities[2]),
        "pr1": (['x','y','z'],quantities[3]),
        "pr0.71": (['x','y','z'],quantities[4]),
        "pr0.2": (['x','y','z'],quantities[5]),
        "pr0.025": (['x','y','z'],quantities[6]),
    },
    coords={
        "x":(["x"],xF),
        "y":(["y"],yF),
        "z":(["z"],zF),
        "time":length[2],
    },

).chunk(1000000)

først loader jeg lige en enkelt ind så jeg bedre kan holde styr på det hele.

In [3]:
quantities, quanList, xF, yF, zF, length, storl, paraString =da.from_array(readDNSdata('../data/raw/field.0494.u'),chunks=10000)


In [14]:
a=quantities[0].dtype
b=quantities[0].shape

xF og zF er 1 kortere end resten. Fjerner den første værdi, og indlæser i et xarray

In [21]:
xF=xF[1:]
zF=zF[1:]

In [22]:
ds=xr.Dataset(
    {
        "u_vel": (['x','y','z'],quantities[0]),
        "v_vel": (['x','y','z'],quantities[1]),
        "w_vel": (['x','y','z'],quantities[2]),
        "pr1": (['x','y','z'],quantities[3]),
        "pr0.71": (['x','y','z'],quantities[4]),
        "pr0.2": (['x','y','z'],quantities[5]),
        "pr0.025": (['x','y','z'],quantities[6]),
    },
    coords={
        "x":(["x"],xF),
        "y":(["y"],yF),
        "z":(["z"],zF),
        "time":length[2],
    },

).chunk(1000)




gemmer den som dens filnavn

In [23]:
ds.to_netcdf('field.0494.nc')


Nu prøver jeg så at bruge dask delayed til at læse alle filerne ind og derefter gemme dem som dask/xarray. Derefter
skal de så gemmes ned på harddisken....
https://docs.dask.org/en/latest/array-creation.html


In [24]:
quantities, quanList, xF, yF, zF, length, storl, paraString =dask.delayed(readDNSdata,nout=8)('../data/raw/field.0494.u')


In [25]:
xF=da.from_delayed(xF,(257,),dtype='float64')
xF=xF[:-1]

In [9]:

utemp=da.from_delayed(quantities[0],(256,201,256),dtype='float64')
vtemp=da.from_delayed(quantities[1],(256,201,256),dtype='float64')
wtemp=da.from_delayed(quantities[2],(256,201,256),dtype='float64')
s1=da.from_delayed(quantities[3],(256,201,256),dtype='float64')
s2=da.from_delayed(quantities[4],(256,201,256),dtype='float64')
s3=da.from_delayed(quantities[5],(256,201,256),dtype='float64')
s4=da.from_delayed(quantities[6],(256,201,256),dtype='float64')


    #saving in a xarray
ds=xr.Dataset(
    {
        "u_vel": (['x','y','z'],utemp),
        "v_vel": (['x','y','z'],vtemp),
        "w_vel": (['x','y','z'],wtemp),
        "pr1": (['x','y','z'],s1),
        "pr0.71":(['x','y','z'],s2),
        "pr0.2": (['x','y','z'],s3),
        "pr0.025": (['x','y','z'],s4),
    },
    coords={
        "x":(["x"],xF),
        "y":(["y"],yF),
        "z":(["z"],zF),
        "time":(["time"],length[2]),
    },

).chunk(1000)

ds.to_netcdf(iterim_path+filename[:-1]+'nc')

ValueError: Could not convert tuple of form (dims, data[, attrs, encoding]): (['time'], Delayed('getitem-6c26ffa6f36358cd1c8d57cb41451102')) to Variable.